## Import Equations of Motion

In [116]:
import sympy as sym
import numpy as np
sym.init_printing()

from ae353_quadrotor import get_gravity
planet = "Earth"
grav = get_gravity(planet, verbose=True)

# Mass and moment of inertia of the bus
mass = 0.5    # The mass of the quadrotor in kg
Ixx = 0.0023  # The moment of inertia of the quadrotor in 
Iyy = 0.0023  # The mass of the quadrotor in kg
Izz = 0.0040  # The mass of the quadrotor in kg
lxy = 0.25    # The x or y distance from the quadrotor frame to the mocap markers in meters
lz = 0.046875 # The z distance from the quadrotor frame to the mocap markers in meters

# Define position
px_inW, py_inW, pz_inW = sym.symbols('p_x, p_y, p_z')
p_inW = sym.Matrix([[px_inW],
                    [py_inW],
                    [pz_inW]])

# Define velocity
vx_inB, vy_inB, vz_inB = sym.symbols('v_x, v_y, v_z')
v_inB = sym.Matrix([[vx_inB],
                    [vy_inB],
                    [vz_inB]])

# Define roll (phi), pitch (theta), and yaw (psi) angles
phi, theta, psi = sym.symbols('phi, theta, psi')

# Define angular velocities
wx_inB, wy_inB, wz_inB = sym.symbols('omega_x, omega_y, omega_z')
w_inB = sym.Matrix([[wx_inB],
                    [wy_inB],
                    [wz_inB]])

# Define roll, pitch, and yaw rotation matrices that describe the bus's orientation in the 
# world frame
Rx = sym.Matrix([[1,            0,             0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi),  sym.cos(phi)]])
Ry = sym.Matrix([[ sym.cos(theta), 0, sym.sin(theta)],
                 [              0, 1,              0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi),  sym.cos(psi), 0],
                 [           0,             0, 1]])
R_ofB_inW = Rz @ Ry @ Rx
R_ofW_inB = R_ofB_inW.T

# Define inputs
taux_inB, tauy_inB, tauz_inB, fz_inB = sym.symbols('tau_x, tau_y, tau_z, f_z')

# Get the net torque vector
tau_inB = sym.Matrix([[taux_inB],
                      [tauy_inB],
                      [tauz_inB]])
tau_inB = sym.simplify(tau_inB)

# Get the gravity force and convert to body coordiantes
grav_inW = sym.Matrix([[0.],
                       [0.],
                       [-mass * grav]])
grav_inB = R_ofW_inB @ grav_inW

# Get the net force vector
f_inB = grav_inB + sym.Matrix([[0.],
                               [0.],
                               [fz_inB]])
f_inB = sym.simplify(f_inB)

# Assemble the moment of inertia matrix
I_inB = sym.Matrix([[Ixx, 0.,  0.],
                    [0.,  Iyy, 0.],
                    [0.,  0.,  Izz]])

v_inW = R_ofB_inW @ v_inB
xyz_dot = sym.simplify(v_inW)
print("px, py, pz rate as a function of orientation and body-fixed velocities:")
sym.N(xyz_dot,3)

v_inB_dot = (1 / mass) * (f_inB - w_inB.cross(mass * v_inB))
v_inB_dot = sym.simplify(v_inB_dot)
print("Body-fixed velocity rates as a function of orientation, body-fixed velocities, and applied forces:")
sym.N(v_inB_dot,3)

# Define the transformation that takes roll, pitch, and yaw rates to body-fixed angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.Matrix.hstack(ex, Rx.T@ey, (Ry@Rx).T@ez)

# Invert the transformation to get a matrix that takes body-fixed angular rates to 
# roll, pitch, and yaw rates
M_inv = sym.simplify(M.inv())

# Convert the body-fixed angular rates to roll, pitch, and yaw rates
rpy_dot = sym.simplify(M_inv@w_inB)
print("Roll, pitch, and yaw rate as a function of orientation and body-fixed angular rates:")
sym.N(rpy_dot,3)

# Apply Euler's equation to get the time derivative of the angular velocities of the
# quadrotor in the quadrotor frame
w_inB_dot = I_inB.inv() @ (tau_inB - w_inB.cross(I_inB@w_inB))
w_inB_dot = sym.simplify(w_inB_dot)
print("Time derivative of the angular velocities in the body-fixed frame:")
sym.N(w_inB_dot, 3)

f = sym.Matrix.vstack(xyz_dot,
                      v_inB_dot,
                      rpy_dot,
                      w_inB_dot)

# Position of markers in body frame
mark1_inB = sym.Matrix([[lxy],
                        [0.],
                        [lz]])
mark2_inB = sym.Matrix([[0.],
                        [lxy],
                        [lz]])
mark3_inB = sym.Matrix([[-lxy],
                        [0.],
                        [lz]])
mark4_inB = sym.Matrix([[0.],
                        [-lxy],
                        [lz]])

# Position of markers in world frame
mark1_inW = p_inW + R_ofB_inW @ mark1_inB
mark2_inW = p_inW + R_ofB_inW @ mark2_inB
mark3_inW = p_inW + R_ofB_inW @ mark3_inB
mark4_inW = p_inW + R_ofB_inW @ mark4_inB

# Sensor model
g = sym.simplify(sym.Matrix.vstack(mark1_inW, mark2_inW, mark3_inW, mark4_inW))


px, py, pz rate as a function of orientation and body-fixed velocities:
Body-fixed velocity rates as a function of orientation, body-fixed velocities, and applied forces:
Roll, pitch, and yaw rate as a function of orientation and body-fixed angular rates:
Time derivative of the angular velocities in the body-fixed frame:


## System Linearization and State Space Form

### Equilbrium

In [143]:
# Defined nonlinear state and input vectors
m = sym.Matrix([px_inW, py_inW, pz_inW,
                vx_inB, vy_inB, vz_inB,
                phi, theta, psi,
                wx_inB, wy_inB, wz_inB])

n = sym.Matrix([taux_inB, tauy_inB, tauz_inB, fz_inB])

# Defined equilibrium point
p_x_e = 0.
p_y_e = 0.
p_z_e = 0.
v_x_e = 0.
v_y_e = 0.
v_z_e = 0.
phi_e = 0.
theta_e = 0.
psi_e = 0.
w_x_e = 0.
w_y_e = 0.
w_z_e = 0.
tau_x_e = 0.
tau_y_e = 0.
tau_z_e = 0.
f_z_e = mass * grav

equilibrium = {px_inW: p_x_e,
               py_inW: p_y_e,
               pz_inW: p_z_e,
               vx_inB: v_x_e,
               vy_inB: v_y_e,
               vz_inB: v_z_e,
               phi: phi_e,
               theta: theta_e,
               psi: psi_e,
               wx_inB: w_x_e,
               wy_inB: w_y_e,
               wz_inB: w_z_e,
               taux_inB: tau_x_e,
               tauy_inB: tau_y_e,
               tauz_inB: tau_z_e,
               fz_inB: f_z_e}

# Nonlinear state m at equilbrium
m_num = sym.lambdify([px_inW, py_inW, pz_inW,
                vx_inB, vy_inB, vz_inB,
                phi, theta, psi,
                wx_inB, wy_inB, wz_inB],  m)

#Evaluate m at the equilibrium point
m_e = m_num(p_x_e, p_y_e, p_z_e, v_x_e, v_y_e, v_z_e, phi_e, theta_e, psi_e, w_x_e, w_y_e, w_z_e)

# Input at equilbirum
n_num = sym.lambdify([taux_inB, tauy_inB, tauz_inB, fz_inB], n)

#Evaluate n at the equilibrium point
n_e = n_num(tau_x_e, tau_y_e, tau_z_e, f_z_e)

# Create a lambda function for o
o_num = sym.lambdify([px_inW, py_inW, pz_inW,
                vx_inB, vy_inB, vz_inB,
                phi, theta, psi,
                wx_inB, wy_inB, wz_inB], g)

# Evaluate o at the equilibrium point
o_e = o_num(p_x_e, p_y_e, p_z_e, v_x_e, v_y_e, v_z_e, phi_e, theta_e, psi_e, w_x_e, w_y_e, w_z_e)

sym.pprint(o_e)

 [[ 0.25    ]
  [ 0.      ]
  [ 0.046875]
  [ 0.      ]
  [ 0.25    ]
  [ 0.046875]
  [-0.25    ]
  [ 0.      ]
  [ 0.046875]
  [ 0.      ]
  [-0.25    ]
 [ 0.046875]]


### Linearize the EoM

In [118]:
# Linearize the equations of motion
A = f.jacobian(m)
B = f.jacobian(n)

A_e = A.subs(equilibrium)
B_e = B.subs(equilibrium)

C = g.jacobian(m)
C_e = C.subs(equilibrium)

D = g.jacobian(n)
D_e = D.subs(equilibrium)

print("Linearized A matrix at equilibrium:")
sym.pprint(A_e)


Linearized A matrix at equilibrium:
⎡0  0  0  1  0  0    0     0    0  0  0  0⎤
⎢                                         ⎥
⎢0  0  0  0  1  0    0     0    0  0  0  0⎥
⎢                                         ⎥
⎢0  0  0  0  0  1    0     0    0  0  0  0⎥
⎢                                         ⎥
⎢0  0  0  0  0  0    0    9.81  0  0  0  0⎥
⎢                                         ⎥
⎢0  0  0  0  0  0  -9.81   0    0  0  0  0⎥
⎢                                         ⎥
⎢0  0  0  0  0  0    0     0    0  0  0  0⎥
⎢                                         ⎥
⎢0  0  0  0  0  0    0     0    0  1  0  0⎥
⎢                                         ⎥
⎢0  0  0  0  0  0    0     0    0  0  1  0⎥
⎢                                         ⎥
⎢0  0  0  0  0  0    0     0    0  0  0  1⎥
⎢                                         ⎥
⎢0  0  0  0  0  0    0     0    0  0  0  0⎥
⎢                                         ⎥
⎢0  0  0  0  0  0    0     0    0  0  0  0⎥
⎢                                       

In [119]:
print("\nLinearized B matrix at equilibrium:")
sym.pprint(B_e)


Linearized B matrix at equilibrium:
⎡       0                 0            0     0 ⎤
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0    2.0⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢434.782608695652         0            0     0 ⎥
⎢                               

print("Linearized C matrix at equilibrium:")
sym.pprint(C_e)

In [120]:
print("Linearized D matrix at equilibrium:")
sym.pprint(D_e)

Linearized D matrix at equilibrium:
⎡0  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎣0  0  0  0⎦


### Verify Controllability and Observability

In [121]:
# Convert A and B to numeric arrays for controllability/observability checks
A_num = np.array(A_e, dtype=float)
B_num = np.array(B_e, dtype=float)
C_num = np.array(C_e, dtype=float)

l = A_num.shape[0]
j = B_num.shape[1]

# Controllability Check
controllability_matrix = B_num
for i in range(1, l):
    controllability_matrix = np.hstack((controllability_matrix, np.linalg.matrix_power(A_num, i) @ B_num))

rank_C = np.linalg.matrix_rank(controllability_matrix)
print(f"\nControllability matrix rank: {rank_C} out of {l}")
if rank_C == l:
    print("System is controllable.")
else:
    print("System is not fully controllable.")

# Observability Check
observability_matrix = C_num
for i in range(1, l):
    observability_matrix = np.vstack((observability_matrix, C_num @ np.linalg.matrix_power(A_num, i)))

rank_O = np.linalg.matrix_rank(observability_matrix)
print(f"\nObservability matrix rank: {rank_O} out of {l}")
if rank_O == l:
    print("System is observable.")
else:
    print("System is not fully observable with chosen C.")


Controllability matrix rank: 12 out of 12
System is controllable.

Observability matrix rank: 12 out of 12
System is observable.


## Gain Selection

### Controller

In [122]:
# Design gain matrices for the controller
import scipy as scipy

Q = np.diag([1/75**2, 1/75**2, 1/7**2, 1/75**2, 1/75**2, 1/70**2, 1/25**2, 1/75**2, 1/75**2, 1/25**2, 1/25**2, 1/25**2])  # State cost
R = np.diag([1/0.265**2, 1/0.265**2, 1/0.7**2, 1/12.9**2])

P = scipy.linalg.solve_continuous_are(A_num, B_num, Q, R)
K = np.linalg.inv(R) @ (B_num.T @ P)

print("\nLQR Gain Matrix K:")
print(K)


LQR Gain Matrix K:
[[-7.19728584e-18 -3.53333333e-03 -3.42583700e-17 -4.98629433e-18
  -6.94909974e-03 -8.73664465e-17  4.97055246e-02 -2.81140462e-17
   4.72105208e-18  1.84663319e-02 -4.24684114e-18  1.28708712e-18]
 [ 3.53333333e-03 -7.77578074e-19  3.11351051e-17  6.86515256e-03
   3.42883671e-20  3.88829645e-18 -4.25878261e-18  4.80956654e-02
   2.15669701e-18 -4.24684114e-18  1.82647218e-02 -3.87467709e-18]
 [ 5.19773536e-18 -2.75943746e-18  1.81801639e-17 -1.40583782e-17
  -5.12469073e-18  3.86235148e-17  2.63205911e-17 -5.90782333e-17
   9.33333333e-03  5.16392733e-18 -1.55456073e-17  2.93030146e-02]
 [-3.12933628e-18  1.01722770e-15  1.84285714e+00 -3.81227683e-18
   1.04650990e-15  1.36997021e+00 -6.16588582e-15  9.30603795e-17
   1.26036459e-16 -9.52335944e-16  4.23842862e-17  1.04936149e-16]]


In [123]:
# Find the eigenvalues of F for stability check
F = A_num - B_num@K

#Determine if all eigenvalues of F have negative real part
if(np.all(np.linalg.eigvals(F).real < 0)):
    print("The controller is asymptotically stable")
else:
    print("The controller is not asymptotically stable")

print(np.linalg.eigvals(F).size)

The controller is asymptotically stable
12


### Observer

In [124]:
# Observer Q and R matrices
Q_o = np.diag([1] * A_num.shape[0]) * 1.2
R_o = np.diag([1] * C_num.shape[0]) * 1.0

P_o = scipy.linalg.solve_continuous_are(A_num.T, C_num.T, np.linalg.inv(R_o), np.linalg.inv(Q_o))

#Solve for L
L =  (Q_o @ C_num @ P_o.T).T

#Define H
H = A_num - L@C_num

print("\nLQR Gain Matrix L:")
print(L)


LQR Gain Matrix L:
[[ 1.65456937e+00 -2.16116706e-16 -2.03428513e-01  1.65456937e+00
  -2.40837083e-16  7.72466607e-17  1.65456937e+00 -2.65557460e-16
   2.03428513e-01  1.65456937e+00 -2.40837083e-16  5.47388541e-17]
 [-2.42985810e-16  1.65456937e+00 -5.35356265e-17 -4.20852662e-16
   1.65456937e+00 -2.03428513e-01 -2.42985810e-16  1.65456937e+00
  -9.89631850e-17 -6.51189575e-17  1.65456937e+00  2.03428513e-01]
 [ 7.05665242e-17  3.70882952e-16  7.57536322e-01 -3.76491043e-16
  -7.61746157e-17  7.57536322e-01  7.05665242e-17 -5.23232183e-16
   7.57536322e-01  5.17624091e-16 -7.61746157e-17  7.57536322e-01]
 [ 5.17948475e+00 -8.77359250e-16 -1.40214405e+00  5.17948475e+00
  -9.29267188e-16  2.46510325e-16  5.17948475e+00 -9.81175126e-16
   1.40214405e+00  5.17948475e+00 -9.29267188e-16  3.59215038e-16]
 [-1.12818103e-15  5.17948475e+00 -4.62488417e-16 -1.87740959e-15
   5.17948475e+00 -1.40214405e+00 -1.12818103e-15  5.17948475e+00
  -9.32297903e-16 -3.78952475e-16  5.17948475e+00  1

In [125]:
if(np.all(np.linalg.eigvals(H).real < 0)):
    print("The observer is asymptotically stable")
else:
    print("The system is not asymptotically stable")

print(np.linalg.eigvals(H))

The observer is asymptotically stable
[-1.45919548+2.20602829j -1.45919548-2.20602829j -1.45919548+2.20602829j
 -1.45919548-2.20602829j -3.07132313+0.j         -3.07132313+0.j
 -1.83842212+0.j         -0.5906448 +0.44593865j -0.5906448 -0.44593865j
 -1.19172317+0.j         -1.00106935+0.j         -1.00106935+0.j        ]


In [ ]:
# Create the Controller class
class Controller():
    def __init__(self):
        """
        Initializes the controller. You should initialize any member variables that
        the controller will use to their starting values. This function is called
        when a new instance of the Controller class is created.
        """
        self.A = A_num 
        self.B = B_num   
        self.C = C_num   
        self.K = K
        self.L = L

        self.m_e = np.array([p_x_e, p_y_e, p_z_e, v_x_e, v_y_e, v_z_e, phi_e, theta_e, psi_e, w_x_e, w_y_e, w_z_e])
        self.n_e = np.array([tau_x_e, tau_y_e, tau_z_e, f_z_e])
        self.o_e = o_e
        self.p_x_e = p_x_e
        self.p_y_e = p_y_e
        self.p_z_e = p_z_e
        self.v_x_e = v_x_e
        self.v_y_e = v_y_e
        self.v_z_e = v_z_e
        self.phi_e = phi_e
        self.theta_e = theta_e
        self.psi_e = psi_e
        self.w_x_e = w_x_e
        self.w_y_e = w_y_e
        self.w_z_e = w_z_e
        self.tau_x_e = tau_x_e
        self.tau_y_e = tau_y_e
        self.tau_z_e = tau_z_e
        self.f_z_e = f_z_e
        self.e_max = 0.4
        self.x_hat_vals = {"p_x" : [],
                     "p_y" : [],
                     "p_z" : []}
        self.phat_history = []


        pass

    
    def reset(self):
        """
        Resets the controller to an initial state. You should reset any variables
        that track states, times, etc. in this function. This function will be 
        called just before the simulation starts running and any time the simulation
        is reset.
        """
        self.xhat = np.zeros((12, 1))  # Initialize state estimate as zero
        self.x_des = np.zeros((12, 1)) # Initialize desired state as zero
        self.xhat_values = {"px" : [], "py" : [], "pz" : [], "vx" : [], "vy" : [], "vz" : [], 
                            "phi" : [], "theta" : [], "psi" : [], "wx" : [], "wy" : [], "wz" : []}
        self.x_hat_vals = {"p_x" : [],
                     "p_y" : [],
                     "p_z" : []}
        self.phat_values = {"p_x" : [],
                     "p_y" : [],
                     "p_z" : []}
        self.desired_values = {"p_des_x" : [], "p_des_y" : [], "p_des_z" : []}
        self.phat_history = []

        pass
    
    
    def run(self, **kwargs):
        """
        Runs the controller. Data is passed from the simulation environment to the
        controller via kwargs, the controller calculates the inputs to the system,
        and then returns them via a return statement
    
        Parameters
        """

        dt = kwargs["dt"]
        self.dt = dt

        # Gate and direction
        next_gate = np.array(kwargs["next_gate"]).reshape(-1, 1)
        dir_gate = np.array(kwargs["dir_gate"]).reshape(-1, 1)
        self.goal = next_gate
        self.goal_direction = dir_gate
        self.phat = self.xhat[:3]

        # Parameters for gradient
        k_des = 1.0
       
        # Calculate distance to the next gate
        distance_to_gate = np.linalg.norm(self.phat - self.goal)
        if distance_to_gate > 14:
            k_attract = 12
        else:
            k_attract = 10

        """
        18.4 s
        distance_to_gate = np.linalg.norm(self.goal_hat - self.goal)
        if distance_to_gate > 13:
            k_attract = 16.5
        else:
            k_attract = 11.5
        """
        
        # Attractive term
        h_attract = k_attract * (self.phat - self.goal) / np.linalg.norm(self.phat - self.goal)

        h = h_attract

        # Desired position update
        if np.linalg.norm(self.goal - self.phat) < self.e_max:
            self.p_des = self.goal
        else:
            self.p_des = self.phat - k_des * h
        self.p_des = self.p_des.reshape(-1, 1)

        # Construct the desired state (x_des)
        zero_vector = np.zeros((9, 1))
        self.x_des = np.vstack((self.p_des, zero_vector))

        # Compute control inputs
        u_des = np.array([self.tau_x_e, self.tau_y_e, self.tau_z_e, self.f_z_e])
        u = -self.K @ (self.xhat - self.x_des)
        inputs = u + u_des.reshape(-1, 1)

        # Measurement update
        o = np.block([kwargs["mocap_1"], kwargs["mocap_2"], kwargs["mocap_3"], kwargs["mocap_4"]]).reshape(-1, 1)
        y = o - self.o_e

        # State estimation update
        self.xhat += dt * (self.A @ self.xhat + self.B @ u - self.L @ (self.C @ self.xhat - y)).astype(float)

        # Extract final control inputs
        torque_x = inputs[0, 0]
        torque_y = inputs[1, 0]
        torque_z = inputs[2, 0]
        force_z = inputs[3, 0]

        # Store historical data
        self.x_hat_vals["p_x"].append(self.xhat[0, 0])
        self.x_hat_vals["p_y"].append(self.xhat[1, 0])
        self.x_hat_vals["p_z"].append(self.xhat[2, 0])
        self.desired_values["p_des_x"].append(self.p_des[0, 0])
        self.desired_values["p_des_y"].append(self.p_des[1, 0])
        self.desired_values["p_des_z"].append(self.p_des[2, 0])

        self.phat_history.append(self.x_des)

        return torque_x, torque_y, torque_z, force_z

In [127]:
# Create an instance of our Controller class to simulate 1 quadrotor
controller1 = Controller()

## Running the Simulation

Before we run the simulation, we must first import the simulation module and then create an instance of the simulator. This is done below:

In [128]:
# Import the quadrotor project. This module is used to simulate, render, and plot a quadrotor dynamic system
import ae353_quadrotor

In [129]:
# Make an instance of the simulator for 1 quadrotor
team_list = [""]
sim = ae353_quadrotor.Quadrotor_Sim(n_quadrotors=1,
                                    team_list=team_list,
                                    planet="Earth",
                                    use_keyboard=True,
                                    visualization=True,
                                    visualization_fr=45.,
                                    animation=False,
                                    verbose=True)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/


In [130]:
# Run the simulation and collect the simulation data\
controllers = [controller1]
(track_stats, data) = sim.run(controllers,
                              sensor_noise=0.01,
                              collect_data=True,
                              max_time=60.0,
                              verbose=True)

PRESS ENTER TO START SIMULATION.
PRESS ESC TO QUIT.
PRESS SPACE TO PAUSE/RESUME SIMULATION.
PRESS BACKSPACE TO RESET SIMULATION.
Termination command detected. Terminating keyboard listener. Goodbye


KeyboardInterrupt: 

In [ ]:
# Print out the track stats for each quadrotor
for i in range(len(track_stats)):
    stat = track_stats[i]
    print("Quadrotor {} completed lap: {}".format(i, stat["completed_lap"]))
    if len(stat["gate_times"]) > 0:
        print("Gate times for Quadrotor {}: {}".format(i, stat["gate_times"]))
    else:
        print("Quadrotor {} passed no gates.".format(i))
    if stat["completed_lap"]:
        print("Lap time for Quadrotor {}: {}".format(i, stat["lap_time"]))
    print("\n")

Quadrotor 0 completed lap: True
Gate times for Quadrotor 0: [3.04, 5.6, 7.99, 10.96, 13.9, 18.92, 20.36]
Lap time for Quadrotor 0: 20.36




## FIXME: Post-processing the simulation data

After the simulation is run, we can plot the data using the Matplotlib package. To start, we import matplotlib and numpy.

In [ ]:
# Import the pyplot module from the matplotlib package
import matplotlib.pyplot as plt

# Import numpy
import numpy as np

### 3D Position Plot

In [147]:
#import necessary modules
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D

#import the quadrotor project
import ae353_quadrotor

# Make an instance of the simulator for 1 quadrotor
team_list = [""]
sim = ae353_quadrotor.Quadrotor_Sim(n_quadrotors=1,
                                    team_list=team_list,
                                    planet="Earth",
                                    use_keyboard=False,
                                    visualization=False,
                                    visualization_fr=45.,
                                    animation=False,
                                    verbose=True)

# Run the simulation and collect the simulation data
(track_stats, data) = sim.run([controller1],
                              sensor_noise=0.01,
                              collect_data=True,
                              max_time=60.0,
                              verbose=True)

#create 3-D plot for collected data
fig = plt.figure(figsize=(7, 7))
ax = plt.axes(projection='3d')

pos = np.array(data[0]['position'])

pos_x_hat = controller1.x_hat_vals["p_x"]
pos_y_hat = controller1.x_hat_vals["p_y"]
pos_z_hat = controller1.x_hat_vals["p_z"]

# Plot ground track
ax.plot3D(pos[:, 0], pos[:, 1], pos[:, 2], color='black', lw = 2.5, label = "Drone Actual Position")
ax.plot3D(pos_x_hat, pos_y_hat, pos_z_hat, 'b--', lw = 2.5, label = "Drone Estimated Position")

#get the gate locations
gates = sim.gate_data['cens']

#plot each gate
for i in range(len(gates)):
    pos = gates[i]
    ax.scatter3D(pos[0], pos[1], pos[2], s = 2500, marker = 's', color='#FF5F05', facecolor = 'none')

# Set labels and titles for ground track
ax.set_xlabel("X Position [meters]", fontsize=12)
ax.set_ylabel("Y Position [meters]", fontsize=12)
ax.set_zlabel("Altitude [meters]", fontsize=12)
ax.legend(bbox_to_anchor=(1.16, 1.05), fancybox=True, shadow=True)
ax.set_box_aspect([1,1,1])  
ax.grid()

plt.tight_layout(pad=2.0)
plt.show()

:0 PASSED GATE 1 IN 3.05s
:0 PASSED GATE 2 IN 5.61s
:0 PASSED GATE 3 IN 8.0s
:0 PASSED GATE 4 IN 10.97s
:0 PASSED GATE 5 IN 13.91s
:0 PASSED GATE 6 IN 18.91s
:0 PASSED GATE 7 IN 20.34s
:0 COMPLETED LAP IN 20.34s


### Race Completion Time

In [ ]:
time_data = []

px_data = []
py_data = []
pz_data = []

px_hat_data = []
py_hat_data = []
pz_hat_data = []

est_act_x = []
est_act_y = []
est_act_z = []

act_des_x = []
act_des_y = []
act_des_z = []

# number of runs
runs = 100
for i in range(runs):
    # resets before every run
    controller1.reset()
    # run the simulation
    controllers = [controller1]
    sim = ae353_quadrotor.Quadrotor_Sim(n_quadrotors=1,
                                        team_list=[""],
                                        planet="Earth",
                                        use_keyboard=False,
                                        visualization=False,
                                        visualization_fr=45.,
                                        animation=False,
                                        verbose=True)
    (track_stats, data) = sim.run(controllers,
                                  sensor_noise=0.01,
                                  collect_data=True,
                                  max_time=60.,
                                  verbose=True)

    time_data.append(track_stats[0]['lap_time'])
    
    position = np.array(data[0]['position'])
    pos_x = position[:,0]
    pos_y = position[:,1]
    pos_z = position[:,2]

    px_data.append(pos_x)
    py_data.append(pos_y)
    pz_data.append(pos_z)
    
    x = np.array([pos_x, pos_y, pos_z]).T

    pos_x_hat = controller1.x_hat_vals["p_x"]
    pos_y_hat = controller1.x_hat_vals["p_y"]
    pos_z_hat = controller1.x_hat_vals["p_z"]
    
    xhat = np.array([pos_x_hat, pos_y_hat, pos_z_hat]).T
    
    err = abs(xhat-x)

    x_des = controller1.desired_values["p_des_x"]
    y_des = controller1.desired_values["p_des_y"]
    z_des = controller1.desired_values["p_des_z"]

    est_act_x.append(np.mean(err[:, :1]))
    est_act_y.append(np.mean(err[:, 1:2]))
    est_act_z.append(np.mean(err[:, 2:3]))

    act_des_x.append(abs(np.mean(x_des - pos_x)))
    act_des_y.append(abs(np.mean(y_des - pos_y)))
    act_des_z.append(abs(np.mean(z_des - pos_z)))

:0 PASSED GATE 1 IN 3.04s
:0 PASSED GATE 2 IN 5.6s
:0 PASSED GATE 3 IN 7.99s
:0 PASSED GATE 4 IN 10.96s
:0 PASSED GATE 5 IN 13.9s
:0 PASSED GATE 6 IN 18.91s
:0 PASSED GATE 7 IN 20.34s
:0 COMPLETED LAP IN 20.34s
:0 PASSED GATE 1 IN 3.05s
:0 PASSED GATE 2 IN 5.61s
:0 PASSED GATE 3 IN 8.0s
:0 PASSED GATE 4 IN 10.98s
:0 PASSED GATE 5 IN 13.92s
:0 PASSED GATE 6 IN 18.92s
:0 PASSED GATE 7 IN 20.37s
:0 COMPLETED LAP IN 20.37s
:0 PASSED GATE 1 IN 3.04s
:0 PASSED GATE 2 IN 5.6s
:0 PASSED GATE 3 IN 7.99s
:0 PASSED GATE 4 IN 10.96s
:0 PASSED GATE 5 IN 13.9s
:0 PASSED GATE 6 IN 18.91s
:0 PASSED GATE 7 IN 20.36s
:0 COMPLETED LAP IN 20.36s
:0 PASSED GATE 1 IN 3.04s
:0 PASSED GATE 2 IN 5.6s
:0 PASSED GATE 3 IN 7.99s
:0 PASSED GATE 4 IN 10.96s
:0 PASSED GATE 5 IN 13.89s
:0 PASSED GATE 6 IN 18.9s
:0 PASSED GATE 7 IN 20.36s
:0 COMPLETED LAP IN 20.36s
:0 PASSED GATE 1 IN 3.04s
:0 PASSED GATE 2 IN 5.6s
:0 PASSED GATE 3 IN 7.99s
:0 PASSED GATE 4 IN 10.97s
:0 PASSED GATE 5 IN 13.9s
:0 PASSED GATE 6 IN 18.9s

In [135]:
plt.hist(time_data,bins=20, color = 'blue', edgecolor = 'black')
plt.xlabel('Time [s]')
plt.ylabel('Number of Simulations')
plt.title('Race Completion Time for 100 simulations') 
plt.show()

### Tracking Error

In [140]:
fig, axs2 = plt.subplots(1, 3, figsize=(12, 4))
axs2[0].hist(est_act_x, bins=20, color='blue', edgecolor='black')
axs2[0].set_title('X Position')
axs2[0].set_xlabel('Error (m)')
axs2[0].set_ylabel('Number of Simulations')

axs2[1].hist(est_act_y, bins=20, color='black', edgecolor='grey')
axs2[1].set_xlabel('Error (m)')
axs2[1].set_title('Y Position')

axs2[2].hist(est_act_z, bins=20, color='grey', edgecolor='black')
axs2[2].set_xlabel('Error (m)')
axs2[2].set_title('Z Position')

fig.suptitle('Error Between Estimated and Actual Position for 100 simulations', fontsize=14)

plt.show()

KeyboardInterrupt: 

In [ ]:
#set the rmse tolerance
act_des_tolerance = 1.5

#calculate the average RMSE in the roll, pitch, and yaw angles
average_act_des_x = np.mean(act_des_x)
average_act_des_y = np.mean(act_des_y)
average_act_des_z = np.mean(act_des_z)

#print out RMSE values
print(f"The average error x position is: {average_act_des_x} meters")
print(f"The average error y position is: {average_act_des_y} meters")
print(f"The average error z position is: {average_act_des_z} meters")

#check if Requirement 1 is met
if (average_act_des_x <= act_des_tolerance and
    average_act_des_y <= act_des_tolerance and 
    average_act_des_z <= act_des_tolerance):
    print('Requirement #2 is satisfied')
else:
    print('Requirement #2 is not satisfied')

The average error between the actual and desired drone x position is: 0.6099074548278918 meters
The average error between the actual and desired drone y position is: 1.2568817841509583 meters
The average error between the actual and desired drone z position is: 1.2500426496242227 meters
Requirement #2 is satisfied
